In [ ]:
from PIL import Image
import numpy as np
from tqdm import tqdm
from vitssm.data.datasets import VideoDataset, NextFrameDatasetWrapper

dataset = VideoDataset(
    data_path="data_store/VMDsprites/folds/train_0.csv",
    num_frames=5,
    image_size=(32, 32),
)

In [3]:
import torch

ckpt = torch.load("checkpoints/vmdsprites-UPT_AE_T-32.pth")

ckpt["model"]

/tmp/ipykernel_2999/3969226773.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("checkpoints/vmdsprites-UPT_AE_T-32.pth")


KeyError: 'model'

In [2]:
from vitssm.models.upt._autoencoder import UPTAE_M, UPTAE_T, UPTAE_S
from vitssm.models.upt._approximator import UPTCoCaA_M
from vitssm.utils import count_parameters
import torch
from einops import rearrange, repeat

ae = UPTAE_M(resolution=32)
app = UPTCoCaA_M(context_length=4)

x = torch.randn(1, 4, 3, 32, 32)

b, f, c, h, w = x.shape
x = rearrange(x, "b f c h w -> (b f) c h w")
output_pos = rearrange(
    torch.stack(torch.meshgrid([torch.arange(32), torch.arange(32)], indexing="ij")),
    "ndim height width -> (height width) ndim",
).float()

x = ae.encode(x)
print(x.shape)
x = rearrange(x, "(b f) t e -> b f t e", b=b, f=f)
x = app(x)
print(x.shape)

print("autoencoder: ", count_parameters(ae))
print("approximator: ", count_parameters(app))
print("total: ", count_parameters(ae) + count_parameters(app))

torch.Size([4, 8, 96])
torch.Size([1, 8, 96])
autoencoder:  1139907
approximator:  484320
total:  1624227


In [1]:
from vitssm.models.dit import DiT_models
from vitssm.utils import count_parameters

model = DiT_models["DiT_M_1"]()
model.initialize_weights()
count_parameters(model)

1057444

In [9]:
from vitssm.models.diffusion import create_diffusion
import torch.nn as nn
import torch

diff = create_diffusion("trailing4")
diff.betas

ImportError: cannot import name 'CaptionProjection' from 'diffusers.models.embeddings' (/home/david/micromamba/envs/vitssm/lib/python3.12/site-packages/diffusers/models/embeddings.py)

In [12]:
import numpy as np
import pandas as pd
import wandb
from vitssm.data import get_dataloaders_next_frame
from vitssm.utils.visual import model_output_to_video
from omegaconf import OmegaConf
import random

config = OmegaConf.load("configs/DIT/dit_tiny_config.yml")
_, eval_loader = get_dataloaders_next_frame(config.dataset)

run = wandb.init(project="test")

for step in range(0, 10, 2):
    rollout_data = []
    roll_steps=[]
    for rollout_step in range(1, 65):
        rollout_data.append(random.randint(1, 10))
        roll_steps.append(rollout_step)
    wandb.log({"ssim": rollout_data, "rollout_step": roll_steps}, step=step)

    #table = wandb.Table(data=rollout_data, columns=["step", "ssim", "psnr"])
    # axes are (time, channel, height, width)
    frames = next(iter(eval_loader))[1]
    run.log(
        {
            #"rollout_metrics": rollout_data,
            #"ssim": wandb.plot.line(table, x="step", y="ssim", title="test over rollout steps"),
            #"psnr": wandb.plot.line(table, x="step", y="test", title="test over rollout steps"),
            "rollout: ground truth vs. prediction": [
                wandb.Video(model_output_to_video(frames[step]),fps=4),
                wandb.Video(model_output_to_video(frames[step+1]), fps=4),
            ],
        }, 
        step=step
    )

run.finish()

In [5]:
model_output_to_video(frames[0]).shape

(16, 32, 32, 3)

In [10]:
run.finish()